In [ ]:
import random
import numpy as np
from pandas import DataFrame
from datetime import datetime, timedelta


In [ ]:
import numpy as np
import pandas as pd
from datetime import timedelta, datetime

np.random.seed(99)
# Parameter simulasi
hours_peak = (9, 12)
hours_offpeak = (12, 18)
hours_night = (18, 24)
service_rate = 1 / (2 / 60)  # Tingkat pelayanan per menit (1 paket tiap 2 menit)
operational_cost_per_hour = 50000
package_revenue = 10000
late_penalty = 5000
target_waiting_time = 15 / 60  # Batas waktu tunggu dalam jam

# Fungsi distribusi kedatangan per periode
def arrival_distribution(hour):
    if hours_peak[0] <= hour < hours_peak[1]:
        return np.random.normal(40, 10)
    elif hours_offpeak[0] <= hour < hours_offpeak[1]:
        return np.random.exponential(20)
    elif hours_night[0] <= hour < hours_night[1]:
        return np.random.uniform(10, 30)
    return 0

# Simulasi
operating_hours = 15
arrival_times, waiting_times, service_times, start_times, end_times = [], [], [], [], []
arrival_timestamps, start_timestamps, end_timestamps = [], [], []
num_in_queue, delays, operational_cost = [], 0, 0
cur_time, last_completion_time, total_denda = 0, 0, 0
queue_start_time = datetime.strptime("09:00", "%H:%M")

for hour in range(operating_hours):
    arrival_rate = arrival_distribution(9 + hour)
    if arrival_rate <= 0:
        continue

    for _ in range(int(arrival_rate)):
        cur_time += np.random.exponential(1 / arrival_rate)
        service_time = np.random.exponential(2 / 60)

        # Konversi waktu dalam bentuk jam
        arrival_time = queue_start_time + timedelta(hours=cur_time)
        waiting_start_time = max(arrival_time, queue_start_time + timedelta(hours=last_completion_time))
        completion_time = waiting_start_time + timedelta(hours=service_time)

        # Catat hasil simulasi
        arrival_times.append(cur_time)
        arrival_timestamps.append(arrival_time.strftime("%H:%M"))
        start_timestamps.append(waiting_start_time.strftime("%H:%M"))
        end_timestamps.append(completion_time.strftime("%H:%M"))

        # Hitung waktu tunggu
        waiting_time = (waiting_start_time - arrival_time).total_seconds() / 60  # dalam menit
        waiting_times.append(waiting_time)

        # Tambahkan denda jika waktu tunggu melebihi batas
        if waiting_time > target_waiting_time * 60:  # konversi ke menit
            delays += 1
            total_denda += late_penalty

        # Data lain
        start_times.append((waiting_start_time - queue_start_time).total_seconds() / 3600)
        end_times.append((completion_time - queue_start_time).total_seconds() / 3600)
        service_times.append(service_time * 60)  # dalam menit
        last_completion_time = (completion_time - queue_start_time).total_seconds() / 3600
        num_in_queue.append(len([t for t in end_times if t > cur_time]))

    operational_cost += operational_cost_per_hour

# DataFrame hasil simulasi
simulation_results = pd.DataFrame({
    'Arrival Time (minutes)': arrival_times,
    'Arrival Timestamp': arrival_timestamps,
    'Service Start Timestamp': start_timestamps,
    'Completion Timestamp': end_timestamps,
    'Service Time (minutes)': service_times,
    'Waiting Time (minutes)': waiting_times,
    'Packages in Queue': num_in_queue,
})

# Hasil simulasi
avg_waiting_time = simulation_results['Waiting Time (minutes)'].mean()
delay_probability = delays / len(arrival_times)
daily_penalty = total_denda
daily_revenue = len(arrival_times) * package_revenue
net_profit = daily_revenue - (operational_cost + daily_penalty)

print("Rata-rata Waktu Tunggu Paket (menit):", avg_waiting_time)
print("Probabilitas Keterlambatan Paket:", delay_probability)
print("Biaya Operasional Harian (Rp):", operational_cost)
print("Total Denda Keterlambatan (Rp):", daily_penalty)
print("Keuntungan Bersih Harian (Rp):", net_profit)

# Tampilkan hasil simulasi
simulation_results


Rata-rata Waktu Tunggu Paket (menit): 20.470611412135924
Probabilitas Keterlambatan Paket: 0.3883495145631068
Biaya Operasional Harian (Rp): 750000
Total Denda Keterlambatan (Rp): 600000
Keuntungan Bersih Harian (Rp): 1740000


,Arrival Time (minutes),Arrival Timestamp,Service Start Timestamp,Completion Timestamp,Service Time (minutes),Waiting Time (minutes),Packages in Queue
0,0.045256,09:02,09:02,09:02,0.063903,0.000000,1
1,0.088041,09:05,09:05,09:06,1.667659,0.000000,1
2,0.097199,09:05,09:06,09:07,0.095642,1.118177,2
3,0.218257,09:13,09:13,09:13,0.013698,0.000000,1
4,0.256331,09:15,09:15,09:18,2.746891,0.000000,1
...,...,...,...,...,...,...,...
304,14.592051,23:35,23:36,23:38,2.241358,0.748349,3
305,14.758401,23:45,23:45,23:49,4.348887,0.000000,1
306,14.879670,23:52,23:52,23:53,1.111744,0.000000,1
307,14.961185,23:57,23:57,23:59,1.633693,0.000000,1


In [ ]:
# Konversi kolom 'Arrival Timestamp' ke tipe waktu
simulation_results['Arrival Timestamp'] = pd.to_datetime(simulation_results['Arrival Timestamp'], format="%H:%M")

# Filter data untuk setiap periode waktu
busy_period_skenario0 = simulation_results[
    (simulation_results['Arrival Timestamp'].dt.hour >= hours_peak[0]) &
    (simulation_results['Arrival Timestamp'].dt.hour < hours_peak[1])
]
offpeak_period = simulation_results[
    (simulation_results['Arrival Timestamp'].dt.hour >= hours_offpeak[0]) &
    (simulation_results['Arrival Timestamp'].dt.hour < hours_offpeak[1])
]
night_period = simulation_results[
    (simulation_results['Arrival Timestamp'].dt.hour >= hours_night[0]) &
    (simulation_results['Arrival Timestamp'].dt.hour < hours_night[1])
]

# Hitung rata-rata waktu tunggu untuk masing-masing periode
avg_waiting_time_busy = busy_period_skenario0['Waiting Time (minutes)'].mean()
avg_waiting_time_offpeak = offpeak_period['Waiting Time (minutes)'].mean()
avg_waiting_time_night = night_period['Waiting Time (minutes)'].mean()

# Tampilkan hasil
print("Rata-rata Waktu Tunggu pada Jam Sibuk (menit):", avg_waiting_time_busy)
print("Rata-rata Waktu Tunggu pada Jam Tidak Sibuk (menit):", avg_waiting_time_offpeak)
print("Rata-rata Waktu Tunggu pada Jam Malam (menit):", avg_waiting_time_night)


Rata-rata Waktu Tunggu pada Jam Sibuk (menit): 30.029254677100273
Rata-rata Waktu Tunggu pada Jam Tidak Sibuk (menit): 19.99251679409283
Rata-rata Waktu Tunggu pada Jam Malam (menit): 9.835624059190032


In [ ]:
# Filter data untuk setiap periode waktu
busy_period_queue = simulation_results[
    (simulation_results['Arrival Timestamp'].dt.hour >= hours_peak[0]) &
    (simulation_results['Arrival Timestamp'].dt.hour < hours_peak[1])
]
offpeak_period_queue = simulation_results[
    (simulation_results['Arrival Timestamp'].dt.hour >= hours_offpeak[0]) &
    (simulation_results['Arrival Timestamp'].dt.hour < hours_offpeak[1])
]
night_period_queue = simulation_results[
    (simulation_results['Arrival Timestamp'].dt.hour >= hours_night[0]) &
    (simulation_results['Arrival Timestamp'].dt.hour < hours_night[1])
]

# Hitung rata-rata panjang antrean untuk masing-masing periode
avg_queue_length_busy = busy_period_queue['Packages in Queue'].mean()
avg_queue_length_offpeak = offpeak_period_queue['Packages in Queue'].mean()
avg_queue_length_night = night_period_queue['Packages in Queue'].mean()

# Tampilkan hasil
print("Rata-rata Panjang Antrean pada Jam Sibuk:", avg_queue_length_busy)
print("Rata-rata Panjang Antrean pada Jam Tidak Sibuk:", avg_queue_length_offpeak)
print("Rata-rata Panjang Antrean pada Jam Malam:", avg_queue_length_night)

Rata-rata Panjang Antrean pada Jam Sibuk: 16.008130081300813
Rata-rata Panjang Antrean pada Jam Tidak Sibuk: 8.772151898734178
Rata-rata Panjang Antrean pada Jam Malam: 5.859813084112149


### Skenario Alternatif 1

In [ ]:
###Skenario Alternatif 1
import numpy as np
import pandas as pd
from datetime import timedelta, datetime

np.random.seed(99)
# Parameter simulasi
hours_peak = (9, 12)
hours_offpeak = (12, 18)
hours_night = (18, 24)
service_rate = 1 / (2 / 60)  # Tingkat pelayanan per menit (1 paket tiap 2 menit)
operational_cost_per_hour = 50000
package_revenue = 10000
late_penalty = 5000
target_waiting_time = 15 / 60  # Batas waktu tunggu dalam jam

# Fungsi distribusi kedatangan per periode
def arrival_distribution(hour):
    if hours_peak[0] <= hour < hours_peak[1]:
        return np.random.normal(40, 10)
    elif hours_offpeak[0] <= hour < hours_offpeak[1]:
        return np.random.exponential(20)
    elif hours_night[0] <= hour < hours_night[1]:
        return np.random.normal(35, 5)
    return 0

# Simulasi
operating_hours = 15
arrival_times, waiting_times, service_times, start_times, end_times = [], [], [], [], []
arrival_timestamps, start_timestamps, end_timestamps = [], [], []
num_in_queue, delays, operational_cost = [], 0, 0
cur_time, last_completion_time, total_denda = 0, 0, 0
queue_start_time = datetime.strptime("09:00", "%H:%M")

for hour in range(operating_hours):
    arrival_rate = arrival_distribution(9 + hour)
    if arrival_rate <= 0:
        continue

    for _ in range(int(arrival_rate)):
        cur_time += np.random.exponential(1 / arrival_rate)
        service_time = np.random.exponential(2 / 60)

        # Konversi waktu dalam bentuk jam
        arrival_time = queue_start_time + timedelta(hours=cur_time)
        waiting_start_time = max(arrival_time, queue_start_time + timedelta(hours=last_completion_time))
        completion_time = waiting_start_time + timedelta(hours=service_time)

        # Catat hasil simulasi
        arrival_times.append(cur_time)
        arrival_timestamps.append(arrival_time.strftime("%H:%M"))
        start_timestamps.append(waiting_start_time.strftime("%H:%M"))
        end_timestamps.append(completion_time.strftime("%H:%M"))

        # Hitung waktu tunggu
        waiting_time = (waiting_start_time - arrival_time).total_seconds() / 60  # dalam menit
        waiting_times.append(waiting_time)

        # Tambahkan denda jika waktu tunggu melebihi batas
        if waiting_time > target_waiting_time * 60:  # konversi ke menit
            delays += 1
            total_denda += late_penalty

        # Data lain
        start_times.append((waiting_start_time - queue_start_time).total_seconds() / 3600)
        end_times.append((completion_time - queue_start_time).total_seconds() / 3600)
        service_times.append(service_time * 60)  # dalam menit
        last_completion_time = (completion_time - queue_start_time).total_seconds() / 3600
        num_in_queue.append(len([t for t in end_times if t > cur_time]))

    operational_cost += operational_cost_per_hour

# DataFrame hasil simulasi
simulation_results = pd.DataFrame({
    'Arrival Time (minutes)': arrival_times,
    'Arrival Timestamp': arrival_timestamps,
    'Service Start Timestamp': start_timestamps,
    'Completion Timestamp': end_timestamps,
    'Service Time (minutes)': service_times,
    'Waiting Time (minutes)': waiting_times,
    'Packages in Queue': num_in_queue,
})

# Hasil simulasi
avg_waiting_time = simulation_results['Waiting Time (minutes)'].mean()
delay_probability = delays / len(arrival_times)
daily_penalty = total_denda
daily_revenue = len(arrival_times) * package_revenue
net_profit = daily_revenue - (operational_cost + daily_penalty)

print("Rata-rata Waktu Tunggu Paket (menit):", avg_waiting_time)
print("Probabilitas Keterlambatan Paket:", delay_probability)
print("Biaya Operasional Harian (Rp):", operational_cost)
print("Total Denda Keterlambatan (Rp):", daily_penalty)
print("Keuntungan Bersih Harian (Rp):", net_profit)

# Tampilkan hasil simulasi
simulation_results


Rata-rata Waktu Tunggu Paket (menit): 50.794737486097155
Probabilitas Keterlambatan Paket: 0.6658291457286433
Biaya Operasional Harian (Rp): 750000
Total Denda Keterlambatan (Rp): 1325000
Keuntungan Bersih Harian (Rp): 1905000


,Arrival Time (minutes),Arrival Timestamp,Service Start Timestamp,Completion Timestamp,Service Time (minutes),Waiting Time (minutes),Packages in Queue
0,0.045256,09:02,09:02,09:02,0.063903,0.000000,1
1,0.088041,09:05,09:05,09:06,1.667659,0.000000,1
2,0.097199,09:05,09:06,09:07,0.095642,1.118177,2
3,0.218257,09:13,09:13,09:13,0.013698,0.000000,1
4,0.256331,09:15,09:15,09:18,2.746891,0.000000,1
...,...,...,...,...,...,...,...
393,14.677631,23:40,01:07,01:13,5.614870,87.072113,50
394,14.700818,23:42,01:13,01:17,4.250017,91.295732,51
395,14.704733,23:42,01:17,01:21,4.026394,95.310882,52
396,14.709076,23:42,01:21,01:22,0.639066,99.076702,53


### Skenario Alternatif 2

In [ ]:
### Skenario Alternatif 2
import numpy as np
import pandas as pd
from datetime import timedelta, datetime

# Menentukan random seed untuk konsistensi hasil
np.random.seed(99)

# Parameter simulasi
hours_peak = (9, 12)
hours_offpeak = (12, 18)
hours_night = (18, 24)
service_rate = 1 / (2 / 60)  # Tingkat pelayanan per menit (1 paket tiap 2 menit)
operational_cost_per_hour = 50000
extra_courier_cost_per_hour = 50000  # Biaya kurir tambahan per jam untuk jam sibuk
package_revenue = 10000
late_penalty = 5000
target_waiting_time = 15 / 60  # Batas waktu tunggu dalam jam

# Fungsi distribusi kedatangan per periode
def arrival_distribution(hour):
    if hours_peak[0] <= hour < hours_peak[1]:
        return np.random.normal(40, 10)
    elif hours_offpeak[0] <= hour < hours_offpeak[1]:
        return np.random.exponential(20)
    elif hours_night[0] <= hour < hours_night[1]:
        return np.random.uniform(10, 30)
    return 0

# Simulasi
operating_hours = 15
arrival_times, waiting_times, service_times, start_times, end_times = [], [], [], [], []
arrival_timestamps, start_timestamps, end_timestamps = [], [], []
num_in_queue, delays, operational_cost = [], 0, 0
cur_time, last_completion_time, last_completion_time_extra, total_denda = 0, 0, 0, 0
queue_start_time = datetime.strptime("09:00", "%H:%M")

for hour in range(operating_hours):
    arrival_rate = arrival_distribution(9 + hour)
    if arrival_rate <= 0:
        continue

    # Tambahkan biaya operasional untuk setiap jam, dan biaya ekstra kurir jika pada jam sibuk
    if hours_peak[0] <= (9 + hour) < hours_peak[1]:
        operational_cost += operational_cost_per_hour + extra_courier_cost_per_hour
    else:
        operational_cost += operational_cost_per_hour

    for _ in range(int(arrival_rate)):
        cur_time += np.random.exponential(1 / arrival_rate)
        service_time = np.random.exponential(2 / 60)

        # Konversi waktu dalam bentuk jam
        arrival_time = queue_start_time + timedelta(hours=cur_time)

        # Jika jam sibuk, gunakan dua kurir untuk menangani paket
        if hours_peak[0] <= (9 + hour) < hours_peak[1]:
            # Pilih kurir dengan waktu penyelesaian lebih cepat
            waiting_start_time = max(arrival_time, queue_start_time + timedelta(hours=min(last_completion_time, last_completion_time_extra)))
            completion_time = waiting_start_time + timedelta(hours=service_time)

            # Update waktu penyelesaian untuk kurir yang dipakai
            if last_completion_time <= last_completion_time_extra:
                last_completion_time = (completion_time - queue_start_time).total_seconds() / 3600
            else:
                last_completion_time_extra = (completion_time - queue_start_time).total_seconds() / 3600
        else:
            # Jika bukan jam sibuk, gunakan satu kurir saja
            waiting_start_time = max(arrival_time, queue_start_time + timedelta(hours=last_completion_time))
            completion_time = waiting_start_time + timedelta(hours=service_time)
            last_completion_time = (completion_time - queue_start_time).total_seconds() / 3600

        # Catat hasil simulasi
        arrival_times.append(cur_time)
        arrival_timestamps.append(arrival_time.strftime("%H:%M"))
        start_timestamps.append(waiting_start_time.strftime("%H:%M"))
        end_timestamps.append(completion_time.strftime("%H:%M"))

        # Hitung waktu tunggu
        waiting_time = (waiting_start_time - arrival_time).total_seconds() / 60  # dalam menit
        waiting_times.append(waiting_time)

        # Tambahkan denda jika waktu tunggu melebihi batas
        if waiting_time > target_waiting_time * 60:  # konversi ke menit
            delays += 1
            total_denda += late_penalty

        # Data lain
        start_times.append((waiting_start_time - queue_start_time).total_seconds() / 3600)
        end_times.append((completion_time - queue_start_time).total_seconds() / 3600)
        service_times.append(service_time * 60)  # dalam menit
        num_in_queue.append(len([t for t in end_times if t > cur_time]))

# DataFrame hasil simulasi
simulation_results = pd.DataFrame({
    'Arrival Time (minutes)': arrival_times,
    'Arrival Timestamp': arrival_timestamps,
    'Service Start Timestamp': start_timestamps,
    'Completion Timestamp': end_timestamps,
    'Service Time (minutes)': service_times,
    'Waiting Time (minutes)': waiting_times,
    'Packages in Queue': num_in_queue,
})

# Hasil simulasi
avg_waiting_time = simulation_results['Waiting Time (minutes)'].mean()
#avg_waiting_time_peak = simulation_results.loc[simulation_results['Arrival Timestamp'].between("09:00", "12:00"), 'Waiting Time (minutes)'].mean()
delay_probability = delays / len(arrival_times)
daily_penalty = total_denda
daily_revenue = len(arrival_times) * package_revenue
net_profit = daily_revenue - (operational_cost + daily_penalty)

print("Rata-rata Waktu Tunggu Paket (menit):", avg_waiting_time)
#print("Rata-rata Waktu Tunggu Paket di Jam Sibuk (menit):", avg_waiting_time_peak)
print("Probabilitas Keterlambatan Paket:", delay_probability)
print("Biaya Operasional Harian (Rp):", operational_cost)
print("Total Denda Keterlambatan (Rp):", daily_penalty)
print("Keuntungan Bersih Harian (Rp):", net_profit)

# Tampilkan hasil simulasi
simulation_results


Rata-rata Waktu Tunggu Paket (menit): 4.464394391154261
Probabilitas Keterlambatan Paket: 0.08737864077669903
Biaya Operasional Harian (Rp): 900000
Total Denda Keterlambatan (Rp): 135000
Keuntungan Bersih Harian (Rp): 2055000


,Arrival Time (minutes),Arrival Timestamp,Service Start Timestamp,Completion Timestamp,Service Time (minutes),Waiting Time (minutes),Packages in Queue
0,0.045256,09:02,09:02,09:02,0.063903,0.000000,1
1,0.088041,09:05,09:05,09:06,1.667659,0.000000,1
2,0.097199,09:05,09:05,09:05,0.095642,0.000000,2
3,0.218257,09:13,09:13,09:13,0.013698,0.000000,1
4,0.256331,09:15,09:15,09:18,2.746891,0.000000,1
...,...,...,...,...,...,...,...
304,14.592051,23:35,23:36,23:38,2.241358,0.748349,3
305,14.758401,23:45,23:45,23:49,4.348887,0.000000,1
306,14.879670,23:52,23:52,23:53,1.111744,0.000000,1
307,14.961185,23:57,23:57,23:59,1.633693,0.000000,1


In [ ]:
# Konversi kolom 'Arrival Timestamp' ke tipe waktu
simulation_results['Arrival Timestamp'] = pd.to_datetime(simulation_results['Arrival Timestamp'], format="%H:%M")

# Filter data untuk setiap periode waktu
busy_period_skenario2 = simulation_results[
    (simulation_results['Arrival Timestamp'].dt.hour >= hours_peak[0]) &
    (simulation_results['Arrival Timestamp'].dt.hour < hours_peak[1])
]
offpeak_period = simulation_results[
    (simulation_results['Arrival Timestamp'].dt.hour >= hours_offpeak[0]) &
    (simulation_results['Arrival Timestamp'].dt.hour < hours_offpeak[1])
]
night_period = simulation_results[
    (simulation_results['Arrival Timestamp'].dt.hour >= hours_night[0]) &
    (simulation_results['Arrival Timestamp'].dt.hour < hours_night[1])
]

# Hitung rata-rata waktu tunggu untuk masing-masing periode
avg_waiting_time_busy = busy_period_skenario2['Waiting Time (minutes)'].mean()
avg_waiting_time_offpeak = offpeak_period['Waiting Time (minutes)'].mean()
avg_waiting_time_night = night_period['Waiting Time (minutes)'].mean()

# Tampilkan hasil
print("Rata-rata Waktu Tunggu pada Jam Sibuk (menit):", avg_waiting_time_busy)
print("Rata-rata Waktu Tunggu pada Jam Tidak Sibuk (menit):", avg_waiting_time_offpeak)
print("Rata-rata Waktu Tunggu pada Jam Malam (menit):", avg_waiting_time_night)


Rata-rata Waktu Tunggu pada Jam Sibuk (menit): 0.822056016802168
Rata-rata Waktu Tunggu pada Jam Tidak Sibuk (menit): 2.8604202843881854
Rata-rata Waktu Tunggu pada Jam Malam (menit): 9.835624059190032


In [ ]:
before = busy_period_skenario0['Waiting Time (minutes)']
after = busy_period_skenario2['Waiting Time (minutes)']

In [ ]:
import numpy as np
from scipy.stats import ttest_ind, ttest_rel

alpha = 0.05

t_stat, p_value = ttest_rel(after, before, alternative='less')
print(f"T-statistic: {t_stat}, P-value: {p_value}")

if p_value < alpha:
    print("Waktu tunggu paket di jam sibuk berkurang secara signifikan")
else:
    print("Waktu tunggu paket di jam sibuk tidak berkurang secara signifikan")

T-statistic: -13.313020510462573, P-value: 7.929609987197456e-26
Waktu tunggu paket di jam sibuk berkurang secara signifikan


### Skenario Alternatif 3

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Set seed untuk konsistensi hasil simulasi
np.random.seed(99)  # Gantilah 42 dengan angka lain jika diperlukan

# Parameter simulasi
hours_peak = (9, 12)
hours_offpeak = (12, 18)
hours_night = (18, 24)
service_rate = 1 / (2 / 60)  # Tingkat pelayanan per menit (1 paket tiap 2 menit)
operational_cost_per_hour = 50000
package_revenue = 10000
late_penalty = 5000
target_waiting_time = 15 / 60  # Batas waktu tunggu dalam jam

# Fungsi distribusi kedatangan per periode
def arrival_distribution(hour):
    if hours_peak[0] <= hour < hours_peak[1]:
        return np.random.normal(40, 10)
    elif hours_offpeak[0] <= hour < hours_offpeak[1]:
        return np.random.exponential(20)
    elif hours_night[0] <= hour < hours_night[1]:
        return np.random.uniform(10, 30)
    return 0

def simulate_hours(operating_hours):
    arrival_times, waiting_times, service_times, start_times, end_times = [], [], [], [], []
    arrival_timestamps, start_timestamps, end_timestamps = [], [], []
    num_in_queue, delays, operational_cost = [], 0, 0
    cur_time, last_completion_time, total_denda = 0, 0, 0
    queue_start_time = datetime.strptime("09:00", "%H:%M")

    for hour in range(operating_hours):
        arrival_rate = arrival_distribution(9 + hour)
        if arrival_rate <= 0:
            continue

        for _ in range(int(arrival_rate)):
            cur_time += np.random.exponential(1 / arrival_rate)
            service_time = np.random.exponential(2 / 60)

            # Konversi waktu dalam bentuk jam
            arrival_time = queue_start_time + timedelta(hours=cur_time)
            waiting_start_time = max(arrival_time, queue_start_time + timedelta(hours=last_completion_time))
            completion_time = waiting_start_time + timedelta(hours=service_time)

            # Catat hasil simulasi
            arrival_times.append(cur_time)
            arrival_timestamps.append(arrival_time.strftime("%H:%M"))
            start_timestamps.append(waiting_start_time.strftime("%H:%M"))
            end_timestamps.append(completion_time.strftime("%H:%M"))

            # Hitung waktu tunggu
            waiting_time = (waiting_start_time - arrival_time).total_seconds() / 60  # dalam menit
            waiting_times.append(waiting_time)

            # Tambahkan denda jika waktu tunggu melebihi batas
            if waiting_time > target_waiting_time * 60:  # konversi ke menit
                delays += 1
                total_denda += late_penalty

            # Data lain
            start_times.append((waiting_start_time - queue_start_time).total_seconds() / 3600)
            end_times.append((completion_time - queue_start_time).total_seconds() / 3600)
            service_times.append(service_time * 60)  # dalam menit
            last_completion_time = (completion_time - queue_start_time).total_seconds() / 3600
            num_in_queue.append(len([t for t in end_times if t > cur_time]))

        operational_cost += operational_cost_per_hour

    # Hasil simulasi
    avg_waiting_time = np.mean(waiting_times) if waiting_times else 0
    delay_probability = delays / len(arrival_times) if arrival_times else 0
    daily_penalty = total_denda
    daily_revenue = len(arrival_times) * package_revenue
    net_profit = daily_revenue - (operational_cost + daily_penalty)

    return avg_waiting_time, delay_probability, operational_cost, daily_penalty, net_profit

# Simulasi 15 jam
avg_waiting_time_15h, delay_probability_15h, operational_cost_15h, daily_penalty_15h, net_profit_15h = simulate_hours(15)

# Simulasi 12 jam
np.random.seed(99)  # Set seed kembali untuk hasil konsisten pada simulasi 12 jam
avg_waiting_time_12h, delay_probability_12h, operational_cost_12h, daily_penalty_12h, net_profit_12h = simulate_hours(12)

# Perbandingan hasil simulasi
print("=== Hasil Simulasi dengan Jam Operasional 15 Jam ===")
print("Rata-rata Waktu Tunggu Paket (menit):", avg_waiting_time_15h)
print("Probabilitas Keterlambatan Paket:", delay_probability_15h)
print("Biaya Operasional Harian (Rp):", operational_cost_15h)
print("Total Denda Keterlambatan (Rp):", daily_penalty_15h)
print("Keuntungan Bersih Harian (Rp):", net_profit_15h)

print("\n=== Hasil Simulasi dengan Jam Operasional 12 Jam ===")
print("Rata-rata Waktu Tunggu Paket (menit):", avg_waiting_time_12h)
print("Probabilitas Keterlambatan Paket:", delay_probability_12h)
print("Biaya Operasional Harian (Rp):", operational_cost_12h)
print("Total Denda Keterlambatan (Rp):", daily_penalty_12h)
print("Keuntungan Bersih Harian (Rp):", net_profit_12h)

=== Hasil Simulasi dengan Jam Operasional 15 Jam ===
Rata-rata Waktu Tunggu Paket (menit): 20.470611412135924
Probabilitas Keterlambatan Paket: 0.3883495145631068
Biaya Operasional Harian (Rp): 750000
Total Denda Keterlambatan (Rp): 600000
Keuntungan Bersih Harian (Rp): 1740000

=== Hasil Simulasi dengan Jam Operasional 12 Jam ===
Rata-rata Waktu Tunggu Paket (menit): 21.855389696273715
Probabilitas Keterlambatan Paket: 0.3780487804878049
Biaya Operasional Harian (Rp): 600000
Total Denda Keterlambatan (Rp): 465000
Keuntungan Bersih Harian (Rp): 1395000
